# This notebook pulls in flight data generated by the TransScrape notebook and combines with weather data so that each flight has a destination airport, information about the departure time and delay if applicable, along with weather readings for the hour that the flight departed. The datasets are merged on an inner join so that only flights with complete weather data are considered. 

In [1]:
import pandas as pd
import os

In [19]:
# Get all flights data from dir
raw_data = pd.read_csv('all_flights.csv')

In [3]:
# Gather only flights departing SFO and get rid of useless columns,
# also adds datetime column for exact departure and one for the 
# hour period in which the flight departed, used for merging with 
# weather data below
def departures_transform(df):
    
    cols_dict = {
    'YEAR': 'Year',
    'DAY_OF_WEEK': 'Weekday',
    'FL_DATE': 'Date',
    'OP_UNIQUE_CARRIER': 'Carrier',
    'TAIL_NUM': 'Tail_Num',
    'OP_CARRIER_FL_NUM': 'Flight_Num',
    'ORIGIN': 'Origin',
    'DEST': 'Dest',
    'CRS_DEP_TIME': 'Dep_Sched',
    'DEP_TIME': 'Dep_Actual',
    'DEP_DELAY_NEW': 'Dep_Delay',
    'CRS_ARR_TIME': 'Arr_Sched'
    }
    
    df = df.loc[df['ORIGIN']=='SFO']
    df = df.drop(columns=
                 ['DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID',
                  'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID',
                  'ARR_TIME', 'ARR_DELAY_NEW','Unnamed: 0', 'Unnamed: 20'])
    df.rename(index=str, columns=cols_dict, inplace=True)
    df.dropna(inplace=True)
    df['Datetime'] = df.apply(
        lambda row: pd.to_datetime(str(row.Date) + '-' + str(int(row.Dep_Sched)).zfill(4)), axis=1)
    df['Hour'] = df.apply(
        lambda row: row.Datetime.replace(minute=0), axis=1)
    
    return df

In [4]:
# Function to get raw weather data and extract useful features
def weather_transform():
    
    weather_dict = {
    'valid': 'Time',
    'tmpf': 'Temp',
    ' dwpf': 'Dewp',
    ' relh': 'Humid',
    ' drct': 'Wind_Dir',
    ' sknt': 'Wind_Sp',
    ' gust': 'Wind_Gust',
    ' p01i': 'Precip',
    ' mslp': 'Pressure',
    ' vsby': 'Visib'
    }
    
    weather = pd.read_csv('sfo_weather.csv')
    weather.drop(weather.filter(like='sky'), axis=1, inplace=True) 
    weather.drop(labels=['station', ' alti', ' wxcodes', ' metar'], axis=1, inplace=True)
    weather.rename(index=str, columns=weather_dict, inplace=True)
    
    weather = weather[weather['Temp']!='M']
    weather = weather[weather['Wind_Gust']!='M']
    weather = weather[weather['Pressure']!='M']
    weather = weather[weather['Wind_Dir']!='M']
    
    
    to_nums = ['Temp', 'Dewp', 'Humid', 'Wind_Dir', 'Precip', 'Pressure', 'Visib']
    weather[to_nums] = weather[to_nums].apply(pd.to_numeric)

    weather['Wind_Sp'] = weather.apply(lambda row: float(row.Wind_Sp) * 1.15078, axis=1)
    weather['Wind_Gust'] = weather.apply(lambda row: float(row.Wind_Gust) * 1.15078, axis=1)

    
    weather['Time'] = weather.apply(
        lambda row: pd.to_datetime(row.Time), axis=1)
    weather['Hour'] = weather.apply(
        lambda row: row.Time.replace(minute=0), axis=1)
    
    
    weather = weather[weather['Time']<'2018-08-01 00:00:00']

    
    return weather




In [5]:
# Merges flight and weather data with an inner join so only flights
# with complete weather readings are considered
def get_departures():
    
    fares_dict = pd.read_pickle('fares_dict.pkl')
    departures = departures_transform(raw_data)
    weather = weather_transform()
    final = pd.merge(departures, weather, how='inner', left_on='Hour', right_on='Hour')
    final['Fare'] = final.apply(lambda row: fares_dict[row.Dest], axis=1)
    
    return departures

In [6]:
departures = get_departures()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
pd.to_pickle(departures, 'departures.pkl')